In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, plot_confusion_matrix, confusion_matrix, plot_roc_curve, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../../data/UsableData.csv")
holdoutdf = pd.read_csv("../../data/UntestableData.csv")
df = df.drop(columns='Unnamed: 0')
df['HvarType'] = df['HvarType'].mask(df['HvarType'] != 'C', other='V')

mask = {
    'C': 0,
    'V': 1
}
df['Target'] = df['HvarType'].map(mask)



In [16]:
notes = df[["Notes"]]
P = []
for x in notes["Notes"]:
    if x == "P" or x == "W" or x == "Y" or x == "Z":
        P.append(True)
    else:
        P.append(False)
D = []
for x in notes["Notes"]:
    if x == "D" or x == "W" or x == "X" or x == "Z":
        D.append(True)
    else:
        D.append(False)
G = []
for x in notes["Notes"]:
    if x == "G" or x == "X" or x == "Y" or x == "Z":
        G.append(True)
    else:
        G.append(False)

notes["P"] = P
notes["D"] = D
notes["G"] = G
df["P"] = notes["P"]
df["D"] = notes["D"]
df["G"] = notes["G"]
df.drop(columns=["Notes"], inplace=True)

<ipython-input-16-ecf6d9d2977b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes["P"] = P
<ipython-input-16-ecf6d9d2977b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes["D"] = D
<ipython-input-16-ecf6d9d2977b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [17]:
df

,Catalog,HIP,Proxy,RAhms,DEdms,Vmag,VarFlag,r_Vmag,RAdeg,DEdeg,AstroRef,Plx,pmRA,pmDE,e_RAdeg,e_DEdeg,e_Plx,e_pmRA,e_pmDE,DE:RA,Plx:RA,Plx:DE,pmRA:RA,pmRA:DE,pmRA:Plx,pmDE:RA,pmDE:DE,pmDE:Plx,pmDE:pmRA,F1,F2,---,BTmag,e_BTmag,VTmag,e_VTmag,m_BTmag,B-V,e_B-V,r_B-V,V-I,e_V-I,r_V-I,CombMag,Hpmag,e_Hpmag,Hpscat,o_Hpmag,m_Hpmag,Hpmax,HPmin,Period,HvarType,moreVar,morePhoto,CCDM,n_CCDM,Nsys,Ncomp,MultFlag,Source,Qual,m_HIP,theta,rho,e_rho,dHp,e_dHp,Survey,Chart,HD,BD,CoD,CPD,(V-I)red,SpType,r_SpType,Target,P,D,G
0,H,2,NaN,00 00 00.91,-19 29 55.8,9.27,NaN,G,0.003797,-19.498837,+,21.90,181.21,-0.93,1.28,0.70,3.10,1.74,0.92,0.12,-0.14,-0.24,-0.29,0.01,0.21,-0.02,-0.19,-0.28,0.14,2.0,1.45,2,10.519,0.033,9.378,0.021,NaN,0.999,0.002,G,1.04,0.00,I,NaN,9.4017,0.0017,0.015,120.0,NaN,9.37,9.44,NaN,C,NaN,NaN,NaN,NaN,NaN,1.0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224690.0,B-20 6688,NaN,NaN,1.04,K3V,4,0,False,False,False
1,H,3,NaN,00 00 01.20,+38 51 33.4,6.61,NaN,G,0.005008,38.859286,NaN,2.81,5.24,-2.91,0.53,0.40,0.63,0.57,0.47,0.06,0.09,0.04,0.43,-0.01,-0.06,0.03,0.24,0.07,0.21,0.0,-0.45,3,6.576,0.004,6.621,0.005,NaN,-0.019,0.004,G,0.00,0.00,H,NaN,6.6081,0.0007,0.008,127.0,NaN,6.60,6.62,NaN,C,NaN,NaN,00000+3852,I,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,224699.0,B+38 5108,NaN,NaN,0.00,B9,S,0,False,False,False
2,H,7,NaN,00 00 05.41,+20 02 11.8,9.64,NaN,G,0.022549,20.036602,NaN,17.74,-208.12,-200.79,1.01,0.79,1.30,1.13,0.82,0.32,0.08,-0.02,-0.04,0.12,0.06,0.11,0.00,0.16,0.43,0.0,0.21,7,10.542,0.039,9.679,0.030,NaN,0.740,0.020,G,0.79,0.02,H,NaN,9.6795,0.0021,0.017,104.0,NaN,9.65,9.72,NaN,C,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B+19 5185,NaN,NaN,0.79,G0,S,0,False,False,False
3,H,8,NaN,00 00 06.55,+25 53 11.3,9.05,3.0,H,0.027292,25.886474,NaN,5.17,19.09,-5.66,1.70,0.93,1.95,1.54,0.88,0.27,-0.66,-0.36,-0.38,-0.12,0.36,-0.21,-0.24,0.32,0.18,0.0,0.98,8,10.433,0.055,9.151,0.029,NaN,1.102,0.051,T,3.92,0.39,O,NaN,8.5522,0.1671,1.460,77.0,NaN,7.15,11.25,327.5,V,1.0,B,NaN,NaN,NaN,1.0,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,224709.0,B+25 5054,NaN,NaN,4.82,M6e-M8.5e Tc,G,1,True,False,False
4,H,9,NaN,00 00 08.48,+36 35 09.4,8.59,NaN,H,0.035342,36.585938,NaN,4.81,-6.30,8.42,0.86,0.55,0.99,1.02,0.65,0.03,0.16,-0.01,0.00,0.07,-0.02,0.08,0.04,0.10,0.13,3.0,-1.26,9,9.962,0.025,8.711,0.015,NaN,1.067,0.023,T,1.03,0.02,L,NaN,8.7534,0.0018,0.014,107.0,NaN,8.73,8.78,NaN,C,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224708.0,B+35 5149,NaN,NaN,1.00,G5,S,0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71617,H,118313,NaN,23 59 49.40,+21 08 59.1,8.94,NaN,H,359.955817,21.149740,NaN,2.32,0.28,-15.77,0.89,0.70,1.15,0.90,0.70,0.28,0.21,0.03,0.02,-0.03,0.06,0.03,-0.25,0.30,0.27,0.0,1.21,118313,9.400,0.019,8.989,0.019,NaN,0.423,0.015,G,0.49,0.02,H,NaN,9.0336,0.0021,0.013,92.0,NaN,9.01,9.06,NaN,C,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224671.0,B+20 5413,NaN,NaN,0.49,F0,S,0,False,False,False
71618,H,118314,NaN,23 59 49.64,-37 51 28.6,9.27,NaN,H,359.956824,-37.857956,NaN,0.93,-6.60,-4.60,1.03,0.76,1.32,1.35,0.85,0.11,-0.10,-0.05,-0.21,-0.03,0.23,-0.06,-0.38,0.00,-0.13,0.0,0.26,118314,10.731,0.034,9.378,0.018,NaN,1.147,0.030,T,1.11,0.03,L,NaN,9.4350,0.0020,0.022,148.0,NaN,9.38,9.47,NaN,C,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C-38 15704,P-38 8593,1.03,K0,S,0,False,False,False
71619,H,118318,NaN,23 59 51.30,+11 40 25.4,6.99,1.0,H,359.963744,11.673709,NaN,1.92,-2.16,2.09,0.78,0.50,0.91,0.88,0.60,0.33,-0.02,-0.09,-0.19,-0.03,0.21,-0.02,-0.11,0.27,0.49,0.0,2.00,118318,9.107,0.015,7.186,0.007,NaN,1.595,0.013,T,1.61,0.01,L,NaN,7.1043,0.0018,0.014,92.0,NaN,7.08,7.

In [50]:
time = df[["RAhms"]]
time["hours"] = time["RAhms"].str.split(" ").str[0].astype(float)
time["minutes"] = time["RAhms"].str.split(" ").str[1].astype(float)
time["seconds"] = time["RAhms"].str.split(" ").str[2].astype(float)
time["hours"] = [x*60*60 for x in time["hours"]]
time["minutes"] = [x*60 for x in time["minutes"]]
time["time"] = time["hours"] + time["minutes"] + time["seconds"]
df["RAhms"] = time["time"]

<ipython-input-50-c4f5b8e92559>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time["hours"] = time["RAhms"].str.split(" ").str[0].astype(float)
<ipython-input-50-c4f5b8e92559>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time["minutes"] = time["RAhms"].str.split(" ").str[1].astype(float)
<ipython-input-50-c4f5b8e92559>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [51]:
x = df.drop(columns=['HvarType','Target'])
y = df['Target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=42)

In [52]:
y_train.value_counts(normalize=True)

0    0.650421
1    0.349579
Name: Target, dtype: float64

In [53]:
x_train

,Catalog,HIP,Proxy,RAhms,DEdms,Vmag,VarFlag,r_Vmag,RAdeg,DEdeg,AstroRef,Plx,pmRA,pmDE,e_RAdeg,e_DEdeg,e_Plx,e_pmRA,e_pmDE,DE:RA,Plx:RA,Plx:DE,pmRA:RA,pmRA:DE,pmRA:Plx,pmDE:RA,pmDE:DE,pmDE:Plx,pmDE:pmRA,F1,F2,---,BTmag,e_BTmag,VTmag,e_VTmag,m_BTmag,B-V,e_B-V,r_B-V,V-I,e_V-I,r_V-I,CombMag,Hpmag,e_Hpmag,Hpscat,o_Hpmag,m_Hpmag,Hpmax,HPmin,Period,moreVar,morePhoto,CCDM,n_CCDM,Nsys,Ncomp,MultFlag,Source,Qual,m_HIP,theta,rho,e_rho,dHp,e_dHp,Survey,Chart,Notes,HD,BD,CoD,CPD,(V-I)red,SpType,r_SpType
9619,H,16113,NaN,12454.90,-39 52 50.0,7.10,1.0,H,51.895426,-39.880562,NaN,2.09,21.01,-2.03,0.52,0.50,0.69,0.58,0.59,-0.04,0.08,0.06,-0.13,0.06,0.07,0.03,0.00,-0.21,0.04,0.0,0.15,16113,9.077,0.012,7.267,0.006,NaN,1.507,0.011,T,1.54,0.02,L,NaN,7.2247,0.0018,0.018,149.0,NaN,7.19,7.25,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,21586.0,NaN,C-40 929,P-40 318,1.52,K4III,2
2767,H,4713,NaN,3632.75,+58 21 50.9,7.12,NaN,H,15.136462,58.364136,NaN,4.52,1.93,-4.79,0.44,0.44,0.73,0.54,0.52,-0.39,0.16,0.03,0.14,0.09,0.00,0.09,0.10,0.13,-0.48,0.0,-2.14,4713,7.354,0.004,7.148,0.004,NaN,0.197,0.005,T,0.22,0.01,L,NaN,7.1890,0.0008,0.010,218.0,NaN,7.17,7.20,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,5813.0,B+57 179,NaN,NaN,0.18,A3,S
43288,H,71410,NaN,52570.51,-03 46 51.7,8.52,NaN,H,219.043792,-3.781015,NaN,4.45,-17.37,-1.84,1.09,0.86,1.31,1.21,1.23,-0.39,-0.11,-0.30,0.16,0.18,-0.12,0.05,-0.65,0.45,-0.49,0.0,0.60,71410,9.961,0.024,8.616,0.013,NaN,1.140,0.021,T,1.10,0.02,L,NaN,8.6947,0.0012,0.011,108.0,NaN,8.68,8.71,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128273.0,B-03 3645,NaN,NaN,0.96,G8III,X
57634,H,95267,H,69777.42,-01 44 26.0,9.47,NaN,H,290.739263,-1.740568,*,0.64,2.11,-3.55,1.15,0.75,1.37,1.24,0.85,0.05,-0.25,-0.19,-0.27,-0.10,0.18,-0.07,-0.45,0.14,0.14,0.0,2.30,95267,11.666,0.106,9.645,0.028,*,0.541,0.107,G,1.58,0.14,S,*,9.5930,0.0033,0.029,93.0,*,9.53,9.63,NaN,NaN,NaN,19230-0144,H,1.0,2.0,C,S,A,AB,155.0,0.142,0.023,1.50,0.62,NaN,G,NaN,182034.0,B-01 3726,NaN,NaN,0.61,K5,G
61419,H,101400,NaN,73980.68,+01 28 04.4,10.72,NaN,H,308.252851,1.467898,NaN,5.53,-2.98,-4.39,1.66,1.51,2.13,2.28,2.09,0.31,0.07,-0.12,0.23,0.17,0.38,0.21,0.53,0.09,0.40,0.0,1.81,101400,12.940,0.357,10.823,0.094,NaN,1.790,0.304,T,1.89,0.15,K,NaN,10.8063,0.0038,0.032,78.0,NaN,10.75,10.87,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,1.89,M0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,H,60704,NaN,44789.83,+54 33 23.3,10.39,NaN,H,186.624290,54.556485,NaN,26.14,14.65,-213.39,1.26,1.18,1.80,1.65,1.22,0.13,0.02,-0.18,0.23,-0.22,0.27,-0.16,0.08,-0.12,0.02,0.0,1.39,60704,11.664,0.076,10.516,0.043,NaN,0.979,0.069,T,1.02,0.06,L,NaN,10.5294,0.0038,0.039,144.0,NaN,10.46,10.62,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238120.0,B+55 1532,NaN,NaN,1.36,K2,S
6265,H,10583,NaN,8172.87,-55 29 36.8,9.52,NaN,G,34.053624,-55.493564,NaN,5.23,-2.94,14.14,0.80,0.87,0.99,0.98,1.11,-0.12,0.09,-0.15,-0.25,0.10,-0.03,0.05,-0.02,-0.02,0.08,0.0,0.40,10583,10.039,0.027,9.549,0.027,NaN,0.456,0.001,G,0.53,0.00,H,NaN,9.6145,0.0017,0.020,94.0,NaN,9.58,9.64,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14229.0,NaN,NaN,P-56 408,0.53,F6/F7Vw...,1
54886,H,90844,NaN,66716.99,-01 00 10.7,5.93,NaN,G,277.987452,-1.002966,NaN,5.72,10.03,-4.63,0.73,0.59,0.82,0.85,0.72,-0.01,-0.20,-0.27,-0.07,-0.17,0.07,-0.11,0.15,-0.18,-0.17,0.0,-1.18,90844,6.137,0.004,5.963,0.005,NaN,0.170,0.018,G,0.22,0.03,C,NaN,5.9863,0.0009,0.007,56.0,NaN,5.98,6.00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,170920.0,B-01 3504,NaN,NaN,0.22,A4III,X
860,H,1485,NaN,

In [ ]:
fun = [1, 2, 3, 4, 5]
fun1 = [2, 6, 3, 8, 5]
fun2 = [5, 2, 7, 5, 2]

In [ ]:
[x*60*60 + y*60 +z for a, x in enumerate(fun) for b, y in enumerate(fun1) for c, z in enumerate(fun2) if a==b and b==c]